In [3]:
import numpy as np
from collections import Counter

In [8]:
from qulacs import QuantumState, QuantumCircuit
from qulacsvis import circuit_drawer
from scipy.linalg import expm
from qulacs.gate import DenseMatrix
from qulacs.gate import X, Y, Z  , Pauli, Identity, merge
from itertools import combinations
import random
# to simulate time evolution
from qulacs import Observable, PauliOperator, StateVector
# to exponentiate
from scipy.linalg import expm

In [13]:
# function for U=exp(-iHt) (a unitary evolution)
# H =(1-gamma)*alpha*H_{prob} + gamma * H_mix
def time_evol_for_mcmc(num_spins:int, H_prob:np.array, H_mix:np.array, 
                        gamma:float, alpha:float,q_state:QuantumState, time_evol:float):
    target_list=list(range(num_spins-1,0,-1))
    H=(1-gamma)*alpha*H_prob + gamma * H_mix
    unitary_mat= np.round(np.expm(-1j*H*time_evol),decimals=6)
    unitary_time_evol_gate=DenseMatrix(target_list,matrix=unitary_mat)
    # update the quantum state
    unitary_time_evol_gate.update_quantum_state(q_state)
    return q_state

In [31]:
a=QuantumState(2)
a.set_computational_basis(0b01)# i might be required to change this in the original code
a.sampling(sampling_count=10)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]